## Preprocessing

### 데이터셋 확인
우선 데이터셋을 직접 하나하나 읽어서 확인 해보면 다음과 같은 사실을 알 수 있습니다.

1. train셋 row 하나 당 2개의 질문과 5개의 답변이 있습니다. row는 644개 입니다.
2. 질문_1은 대부분 반말로 되어있고, 질문_2는 존댓말로 되어있습니다.
3. test셋의 질문은 대부분 존댓말로 이루어져 있고, 두 질문이 이어져 있는 형태입니다.

### 전략 1
GPT-4를 사용해 각 질문에 맞는 답변을 **선택** 하고, test셋과 같이 두개의 질문이 이어져이 있는 형태의 질문_3과 답변을 **생성** 합니다.

1. 질문1에 대한 답변 선택
2. 질문2에 대한 답변 선택
3. 질문3을 만들고, 답변을 참고해서 질문 3에 대한 답변 생성

여기서 질문 3은 test셋 과 같이. 두개의 질문이 이어져 있는 형태의 질문

이렇게 하면 644 x 3 = 1932개의 데이터셋을 만들 수 있습니다.

우선 필요한 라이브러리를 설치합니다.

In [ ]:
!pip3 install openai
!pip3 install pandas

In [19]:
import pandas as pd

df = pd.read_csv('datasets/train.csv')

df.head()

,id,질문_1,질문_2,category,답변_1,답변_2,답변_3,답변_4,답변_5
0,TRAIN_000,면진장치가 뭐야?,면진장치에 사용되는 주요 기술은 무엇인가요?,건축구조,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...,"면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을...",면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여...,면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 ...,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...
1,TRAIN_001,내진설계의 종류 좀 알려줘,내진설계에는 어떤 종류가 있는지 자세히 알려주실 수 있나요?,건축구조,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.","내진설계에는 내진구조, 제진구조, 면진구조가 있습니다. 내진구조는 건물 구조물이 지...","내진설계에는 주로 내진구조, 제진구조, 면진구조의 세 가지 종류가 있습니다. 이들은...","내진설계에는 주로 내진구조, 제진구조, 면진구조가 사용됩니다. 내진구조는 건물 구조...","내진 설계에는 다양한 종류가 있지만, 대표적으로 내진구조, 제진구조, 면진구조가 있..."
2,TRAIN_002,철골구조의 장점이 뭐야?,철골구조의 장점을 알려줘?,건축구조,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...,철골구조의 장점은 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건...,철골구조의 장점은 건물의 외벽에 하중이 적게 걸리기 때문에 고층 건물의 건축이 용이...,"철골구조의 장점은 건물의 외벽이 하중이 걸리지 않아 공간 활용이 용이하고, 고층 건...",철골구조의 장점은 건물의 외벽에 하중이 크게 걸리지 않아 고층 건물을 건축할 수 있...
3,TRAIN_003,철골철근 콘크리트 구조가 뭐야?,철골철근 콘크리트 구조의 장점과 단점에는 무엇이 있을까요?,건축구조,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ...","철골철근콘크리트 구조는 건축물을 지탱하는 주요 구조물인 철골과 철근, 그리고 콘크리...",철골철근 콘크리트 구조는 건축물을 지탱하기 위한 구조물에서 일반적으로 사용되는 방식...,"철골철근콘크리트 구조는 철골과 철근, 그리고 콘크리트를 함께 사용하여 만들어지는 건...","철골철근 콘크리트 구조는 강철 골조와 강철 철근, 그리고 콘크리트를 함께 사용하여 ..."
4,TRAIN_004,철골구조는 어떤 방식이 있어?,철골구조의 다양한 방식이 무엇인가요?,건축구조,철골구조는 일반철골구조와 경량철골구조가 있습니다.,철골구조는 일반철골구조와 경량철골구조가 있습니다. 일반철골구조는 주로 대형 건물이나...,철골구조는 주로 일반철골구조와 경량철골구조로 나뉘어집니다. 이들은 건축 시스템에 따...,철골구조는 주로 일반철골구조와 경량철골구조로 구분됩니다. 이외에도 최근에는 고층 건...,철골구조는 일반철골구조와 경량철골구조 두 가지 방식이 주로 사용됩니다. 일반철골구조...


작업의 편의를 위해 jsonl 파일로 저장합니다.

In [23]:
# 각 작업을 수행한 후 json 파일로 저장
import json

# 결과를 저장할 딕셔너리 리스트 초기화
data_list = []

# DataFrame 반복
for index, row in df.iterrows():
    # 행 데이터를 딕셔너리로 변환
    row_data = row.to_dict()
    # 딕셔너리 리스트에 추가
    data_list.append(row_data)

# 딕셔너리 리스트를 JSON 파일로 저장
with open('datasets/train.jsonl', 'w') as json_file:
    for data in data_list:
        json_file.write(json.dumps(data, ensure_ascii=False))
        json_file.write('\n')

GPT-4 API 를 호출하여 데이터를 전처리합니다.

In [ ]:
from openai import OpenAI

client = OpenAI(
  api_key='YOUR_API_KEY',
)

# JSON 파일을 딕셔너리 리스트로 읽기
data_list = []
with open('datasets/train.jsonl', 'r') as json_file:
    for line in json_file:
        data = json.loads(line)
        data_list.append(data)

instructions = '''
당신은 전문 건축업자 입니다. 3가지 질문에 대한 답을 진행해 주세요. \n는 각 질문의 마지막에만 사용해 주세요.

1. Q1에 대한 답변을 보기에서 선택해 주세요.
2. Q2에 대한 답변을 보기에서 선택해 주세요.
3. Q1과 Q2를 바탕으로 두 가지를 질문하는 질문을 만들고, 그에 대한 답변을 보기를 참고해서 생성해 주세요.

ex)
### 입력
Q1. 면진장치가 뭐야?
Q2. 면진장치에 사용되는 주요 기술은 무엇인가요?
<보기>
A1. 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 격리장치입니다.
A2. 면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을 줄여주는 장치입니다. 주로 지진이나 기타 지반의 진동으로 인한 피해를 방지하기 위해 사용됩니다.
A3. 면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여주는 장치를 말합니다. 이를 통해 건물의 안전성과 안정성을 향상시키고, 지진 등의 외부 충격으로부터 보호하는 역할을 합니다. 지진으로 인한 건물의 피해를 최소화하기 위해 주로 사용됩니다.
A4. 면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 최소화해 주는 진동 격리장치입니다. 이를 통해 건물 내부의 진동을 줄이고 안정성을 유지하는 데 도움을 줍니다.
A5. 면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동을 줄여주는 장치입니다. 지반으로부터 오는 진동 에너지의 영향을 완화시키기 위해 사용됩니다.

### 출력
1. A1
2. A3
3. Q: 면진 장치의 작동 원리를 간단하게 설명해 주세요. 그리고 면진 장치가 건물에 미치는 영향은 무엇인가요? A: 면진 장치는 지반에서 발생하는 진동 에너지를 흡수하거나 격리하여 건물로의 진동 전달을 줄이는 원리로 작동합니다. 이러한 기술은 건물의 기초와 주요 구조 부분 사이에 설치되어, 지진이나 다른 진동 원인으로부터 발생하는 에너지를 효과적으로 분산시키거나 흡수합니다. 이로 인해 건물은 지진과 같은 외부 충격에 더 잘 견딜 수 있게 되며, 구조적 손상과 인명 피해의 위험을 현저히 줄일 수 있습니다. 면진 장치를 통해 건물의 안전성과 안정성이 향상되며, 지진으로 인한 잠재적 피해를 최소화할 수 있습니다.
'''

for i in range(605, len(data_list)): # 실패 시 i부터 시작
    print(f"start {i}th question")
    Q1 = data_list[i]['질문_1']
    Q2 = data_list[i]['질문_2']
    A1 = data_list[i]['답변_1']
    A2 = data_list[i]['답변_2']
    A3 = data_list[i]['답변_3']
    A4 = data_list[i]['답변_4']
    A5 = data_list[i]['답변_5']

    # 변수 맵핑
    answers = {
        "A1": A1,
        "A2": A2,
        "A3": A3,
        "A4": A4,
        "A5": A5
    }

    prompt = f'''
Q1. {Q1}
Q2. {Q2}
<보기>
A1. {A1}
A2. {A2}
A3. {A3}
A4. {A4}
A5. {A5}
'''
    
    max_try = 3
    for j in range(0, max_try):
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "system", "content": f"{instructions}"}, {"role": "user", "content": f"{prompt}"}],
            )
        try:
            output = response.choices[0].message.content
            print(output)
            # 각 작업마다 jsonl 파일로 저장

            output_dict1 = {
                "input": Q1,
                "output": answers[output.split("\n")[0].split(". ")[1]]
            }

            output_dict2 = {
                "input": Q2,
                "output": answers[output.split("\n")[1].split(". ")[1]]
            }

            output_dict3 = {
                "input": f"{output.split('Q: ')[1].split(' A: ')[0]}",
                "output": output.split("\n")[2].split(" A: ")[1]
            }

            # 딕셔너리 리스트를 JSON 파일로 저장
            with open(f'datasets/labeled_train.jsonl', 'a') as json_file:
                json_file.write(json.dumps(output_dict1, ensure_ascii=False))
                json_file.write('\n')
                json_file.write(json.dumps(output_dict2, ensure_ascii=False))
                json_file.write('\n')
                json_file.write(json.dumps(output_dict3, ensure_ascii=False))
                json_file.write('\n')
            break
        except:
            print(f"retry {j}th")

약 50$의 비용이 발생했습니다.

In [47]:
with open(f'datasets/labeled_train.jsonl', 'r') as json_file:
    print(len(json_file.readlines())) # 한 줄 누락됨..

with open(f'datasets/labeled_train.jsonl', 'r') as json_file:
    for line in json_file:
        print(line)
        break

1931
{"input": "면진장치가 뭐야?", "output": "면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여주는 장치를 말합니다. 이를 통해 건물의 안전성과 안정성을 향상시키고, 지진 등의 외부 충격으로부터 보호하는 역할을 합니다. 지진으로 인한 건물의 피해를 최소화하기 위해 주로 사용됩니다."}



## 전략 2
같은 category 내의 임의의 질의 응답을 선택하여, 두 질문을 바탕으로 질문 3을 생성하고, 답변을 생성합니다.

프롬프트에 예시로 Q1, Q2를 서로 비교하는 질의응답을 입력하여, 모델이 예시를 바탕으로 답변을 생성하도록 합니다.

In [69]:
import google.generativeai as genai
genai.configure(api_key="YOUR_API_KEY")
model = genai.GenerativeModel('gemini-pro')

In [74]:
import random

data_set = {}
total_data_list = []

with open(f'datasets/train.jsonl', 'r') as json_file:
    for line in json_file:
        data = json.loads(line)
        total_data_list.append(data)
        if data['category'] not in data_set:
            data_set[data['category']] = []
        else:
            data_set[data['category']].append(data)

print(data_set.keys())

prompt = '''
다음 두 질의응답을 바탕으로 하나의 질의 응답을 만들어 주세요.

ex) 
### 입력
Q1: 창호 결로를 해결하기 위한 가장 효과적인 방법은 무엇인가요?
A1: 창호 결로를 예방하기 위한 대책은 KS에 규정된 프레임을 선택하고, 열관류율이 적은 유리를 선택하는 것이 좋습니다. 또한, 창호 결로 발생 시험을 실시하여 안전한 제품을 선택하는 것이 중요합니다. 또한, 주기적인 창문 청소와 관리를 통해 습기가 축적되는 것을 방지하는 것이 도움이 될 수 있습니다.

Q2: AD, PD에 면한 벽체 결로에 대한 대책은 어떤 것이 있나요?
A2: AD, PD에 면한 벽체 결로에 대한 대책은 단열재를 미실하게 시공하여 결로가 생기는 벽체의 표면 온도를 노점온도 이상으로만 유지해주는 것이 중요합니다. 그 외에도 실외습기의 유입을 차단하고, 적절한 환기 시스템을 활용하여 공간 내 습도를 유지하고 벽체 표면을 건조하게 유지하는 것이 필요합니다. 또한, 외피의 방수 및 수증기 차단 기능을 강화하여 벽체 내부로의 습기 유입을 최소화해야 합니다. 일반적으로는 벽체의 표면뿐만 아니라 내부적으로도 겉지문 및 방수층의 철저한 시공이 필요합니다.

### 출력
Q: 창호 결로와 면한 결로는 어떤 차이가 있나요?
A: 창문에 물이 맺히는 것과 벽에 물이 맺히는 것은 다릅니다. 창문에 물이 맺히는 것은 따뜻한 공기가 차가운 창문을 만나서 생기는데, 이를 막으려면 따뜻한 공기가 창문을 통해 쉽게 나가지 않도록 좋은 창문을 고르는 것이 중요해요. 벽에 물이 맺히는 것은 벽이 충분히 따뜻하지 못하거나, 집안의 습한 공기가 밖으로 잘 나가지 못해서 생깁니다. 이를 해결하기 위해서는 벽을 따뜻하게 유지하고, 집안의 공기가 잘 통하도록 만들어야 해요. 이 두 가지 문제 모두 집을 짓거나 관리할 때 올바른 자재를 고르고, 잘 짓는 것이 중요합니다.
'''
for i in range(0, len(total_data_list)):
    print(f"start {i}th question")

    Q1 = total_data_list[i]['질문_2']
    A1 = total_data_list[i]['답변_3']

    # 같은 카테고리의 다른 질문을 랜덤으로 선택
    category = total_data_list[i]['category']
    print(category)
    random_question = random.choice(data_set[category])
    Q2 = random_question['질문_2']
    A2 = random_question['답변_3']

    text = f'''
### 입력
Q1: {Q1}
A1: {A1}

Q2: {Q2}
A2: {A2}

### 출력'''
    
    print(f"{prompt}{text}")

    result = model.generate_content(f"{prompt}{text}")
    print(result.text)
    break


dict_keys(['건축구조', '기타', '마감재', '마감하자', '시공', '인테리어', '타 마감하자'])
start 0th question
건축구조

다음 두 질의응답을 바탕으로 하나의 질의 응답을 만들어 주세요.

ex) 
### 입력
Q1: 창호 결로를 해결하기 위한 가장 효과적인 방법은 무엇인가요?
A1: 창호 결로를 예방하기 위한 대책은 KS에 규정된 프레임을 선택하고, 열관류율이 적은 유리를 선택하는 것이 좋습니다. 또한, 창호 결로 발생 시험을 실시하여 안전한 제품을 선택하는 것이 중요합니다. 또한, 주기적인 창문 청소와 관리를 통해 습기가 축적되는 것을 방지하는 것이 도움이 될 수 있습니다.

Q2: AD, PD에 면한 벽체 결로에 대한 대책은 어떤 것이 있나요?
A2: AD, PD에 면한 벽체 결로에 대한 대책은 단열재를 미실하게 시공하여 결로가 생기는 벽체의 표면 온도를 노점온도 이상으로만 유지해주는 것이 중요합니다. 그 외에도 실외습기의 유입을 차단하고, 적절한 환기 시스템을 활용하여 공간 내 습도를 유지하고 벽체 표면을 건조하게 유지하는 것이 필요합니다. 또한, 외피의 방수 및 수증기 차단 기능을 강화하여 벽체 내부로의 습기 유입을 최소화해야 합니다. 일반적으로는 벽체의 표면뿐만 아니라 내부적으로도 겉지문 및 방수층의 철저한 시공이 필요합니다.

### 출력
Q: 창호 결로와 면한 결로는 어떤 차이가 있나요?
A: 창문에 물이 맺히는 것과 벽에 물이 맺히는 것은 다릅니다. 창문에 물이 맺히는 것은 따뜻한 공기가 차가운 창문을 만나서 생기는데, 이를 막으려면 따뜻한 공기가 창문을 통해 쉽게 나가지 않도록 좋은 창문을 고르는 것이 중요해요. 벽에 물이 맺히는 것은 벽이 충분히 따뜻하지 못하거나, 집안의 습한 공기가 밖으로 잘 나가지 못해서 생깁니다. 이를 해결하기 위해서는 벽을 따뜻하게 유지하고, 집안의 공기가 잘 통하도록 만들어야 해요. 이 두 가지 문제 모두 집을 짓거나 관리할 때 올바른 자재를 고르고, 잘 짓는 것이 중요합니다.